## List of tables:

### 1. [Table 2: Background characteristics of subjects](#balance)

#### Imports libraries

In [1]:
import matplotlib.pyplot as plt  # Plotting library
import os  # File system handling
import pandas as pd  # Dataframe handling
import statsmodels.formula.api as smf  # Regressions analysis
 
from statsmodels.iolib.summary2 import summary_col  # Regression output table

#### Set project directory

In [2]:
PROJECT_FOLDER = os.path.dirname(os.getcwd())
FINAL_DATA_FOLDER = os.path.join(PROJECT_FOLDER, 'data', 'final')
TABLES_FOLDER = os.path.join(PROJECT_FOLDER, 'reports', 'tables')
FIGURES_FOLDER = os.path.join(PROJECT_FOLDER, 'reports', 'figures')

#### Pandas settings 

In [3]:
pd.set_option("display.precision", 3)
pd.set_option("display.expand_frame_repr", False)
pd.set_option("display.max_rows", 25)

#### Set plotting style

In [4]:
plt.style.use('classic')

#### Set plotting properties 

In [5]:
bar_kw = dict(kind='bar', color='0.4', alpha=0.8, lw=0.5, width=0.7)
line_kw = dict(kind='line', lw=1, alpha=1, legend=True)
font_kw = dict(fontsize=11, color='k')
grid_kw = dict(linewidth=1, axis="y", zorder=2, antialiased=True)
x_lab_kw = dict(fontsize=11, labelpad=3)
y_lab_kw = dict(fontsize=11, labelpad=3)
error_kw = dict(elinewidth=2, ecolor='0.15')
legend_kw = dict(frameon=False)
tick_kw = dict(size=5,
               which='both',
               direction='out',
               right=False,
               top=False,
               labelbottom=True)

#### Retrieving dataframe

In [6]:
DATA = os.path.join(
    FINAL_DATA_FOLDER, 
    'fair_advice_final.feather'
)

df = pd.read_feather(DATA)
df.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3598 entries, 0 to 3597
Data columns (total 28 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   Dataset                   3598 non-null   int64         
 1   SubmitTime                3598 non-null   datetime64[ns]
 2   Duration                  3598 non-null   int64         
 3   Controlquestion           1799 non-null   float64       
 4   Controlquestion_attempts  1799 non-null   float64       
 5   Treatment                 3598 non-null   int64         
 6   Treatment_name            3598 non-null   object        
 7   Dictator                  3598 non-null   int64         
 8   Decision                  3598 non-null   int64         
 9   Advise                    1799 non-null   float64       
 10  Follow                    1799 non-null   float64       
 11  Belief1                   598 non-null    float64       
 12  Belief2             

#### Table 1: Treatment table <a id='treatment'></a>

In [7]:
treat_table = pd.crosstab(df['Risk_type'], df['Advice_type'], margins=True)\
    .rename({0: 'No', 1:'Yes'}, axis=0)\
    .rename(lambda x: x.capitalize(), axis=1)\
    .rename_axis('Advice type:', axis=1)\
    .rename_axis('Risk', axis=0)\
    .rename_axis('Advice', axis=1)
    
treat_table.style.to_latex(
    os.path.join(TABLES_FOLDER, 'treat_table.tex'), 
)

display(treat_table)

Advice,Base,Binding,Free,All
Risk,,,,
No Risk,607,601,603,1811
Risk,600,596,591,1787
All,1207,1197,1194,3598


#### Table 1:  Background characterisitcs of subjects <a id='balance'></a>

In [8]:
sel = (df.Risk == 0)

balance = pd.pivot_table(
    df[sel],
    index='Treatment_name' ,
    aggfunc=['mean', 'sem'],
    values=['Age', 'Female', 'Education', 'Political'],
    margins=True
    )\
    .rename({'mean':'Mean', 'sem':'Se'}, level=0, axis=1)\
    .rename(lambda x: x.capitalize(), axis=1)\
    .rename_axis('Treatment')\
    .swaplevel(axis=1)\
    .sort_index(axis=1, level=0)\
    .round(2)

balance.loc[:, 'N'] = df[sel].groupby('Treatment_name').size()
balance.loc['All', 'N'] = df[sel].groupby('Treatment_name').size().sum()

path = os.path.join(TABLES_FOLDER, 'balance_table.tex')
balance.style.format(precision=2).to_latex(path)

display(balance)

Age       Education       Female       Political             N
                  Mean    Se      Mean    Se   Mean    Se      Mean    Se        
Treatment                                                                        
No Risk Advise   35.53  0.46      4.18  0.05   0.48  0.02     -0.43  0.05   603.0
No Risk Dictate  34.97  0.44      4.11  0.06   0.47  0.02     -0.49  0.04   607.0
No Risk Force    35.42  0.47      4.19  0.05   0.50  0.02     -0.46  0.05   601.0
All              35.30  0.26      4.16  0.03   0.49  0.01     -0.46  0.03  1811.0

#### Check covariate balance using OLS<a id='balance_ols'></a>

In [9]:
ols_results = [
    smf.ols(formula=f'{variable} ~ C(Treatment_name)', data=df).fit(cov_type='HC1')
    for variable in ['Age', 'Female', 'Education', 'Political']
]

auxiliary = {
    'N': lambda x: f'{x.nobs:.0f}',
    'F': lambda x: f'{x.fvalue:.3f}', 
    'P(>F)': lambda x: f'{x.f_pvalue:.2f}',
}

ols_balance = summary_col(
    ols_results,
    stars=True,
    info_dict=auxiliary,
    float_format='%.3f'
)

label_index = {
    'Intercept':'Constant', 
}

ols_balance = ols_balance.tables[0]\
    .rename(mapper=lambda x: x.replace('C(Treatment_name)[T.', '').replace(']', ''), axis=0)\
    .rename(label_index, axis=0)\
    .rename(lambda x: x.capitalize(), axis=1)\
    .rename_axis('Dependent variable:', axis=1)

path = os.path.join(TABLES_FOLDER, 'ols_balance.tex')
ols_balance.style.format(precision=3).to_latex(path)

display(ols_balance)

Dependent variable:,Age,Female,Education,Political
Constant,35.534***,0.484***,4.176***,-0.428***
,(0.465),(0.020),(0.054),(0.046)
No Risk Dictate,-0.567,-0.010,-0.069,-0.060
,(0.640),(0.029),(0.078),(0.064)
No Risk Force,-0.118,0.017,0.017,-0.035
,(0.658),(0.029),(0.076),(0.065)
Risk Advise,-0.148,-0.000,0.019,-0.063
,(0.637),(0.029),(0.077),(0.064)
Risk Dictate,0.694,-0.006,-0.022,0.008
,(0.669),(0.029),(0.079),(0.066)


#### Convert to html

In [10]:
!jupyter nbconvert --output-dir='./docs' --to html 2_balance_and_sample.ipynb

[NbConvertApp] Converting notebook 2_balance_and_sample.ipynb to html
[NbConvertApp] Writing 596705 bytes to docs/2_balance_and_sample.html
